In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('E:/Jupyter Notebook/인공지능인문학/텍스트 윤리검증 데이터/discrimination_data.csv')
df

,Unnamed: 0,text,types,intensity
0,0,남자들은 도대체 왜 그러는걸까?,DISCRIMINATION,1.0
1,1,말하면서 침 개많이 튀길것 같아.,DISCRIMINATION,3.0
2,2,머리가 멍청해서 못읽는거아니냐?,DISCRIMINATION,3.0
3,3,"동유럽, 러시아 애들은 가난해도 사랑으로 결혼하는데 헬조선은 자본주의의 지옥 그 자체다",DISCRIMINATION,1.0
4,4,원래 남자는 365일 발정기야,DISCRIMINATION,2.0
...,...,...,...,...
4430,4430,조상님한테 제사도 안 지내고. 넌 복 받을 게 없겠다.,DISCRIMINATION,1.0
4431,4431,여자 성격이 그렇게 깐깐해서 시집은 가겠냐? ㅋㅋㅋ,DISCRIMINATION,1.0
4432,4432,이슬람 애들 한 번 챙겨줬다가 유럽 꼴 날까 봐 무섭다,DISCRIMINATION,1.0
4433,4433,나도. 이슬람 애들은 왜 그렇게 기를 쓰고 남의 나라 차지하려는지 모르겠어,DISCRIMINATION,1.0


In [3]:
# df = df['text']
# df.to_csv('discrimination_data.csv', index=False)

# Tokenization (한글 형태소 단위로 쪼개기)

In [4]:
import konlpy
import re

# tokenization 함수를 만들어둠
def tokenize_korean_text(text):
    # text = re.sub(r'[^,.?!\w\s]','', text)  
    ## ,.?!와 문자+숫자+_(\w)와 공백(\s)만 남김  # 앞에 r을 붙여주면 deprecation warning이 안뜸 (raw string으로 declare)
    
    okt = konlpy.tag.Okt()
    Okt_morphs = okt.pos(text)   # stem=True로 설정하면 동사원형으로 바꿔서 return
    
    words = []
    for word, pos in Okt_morphs:
        if  pos == 'Adjective' or pos == 'Verb' or pos == 'Noun' or pos == 'Adverb':  # 이 경우에는 형용사, 동사, 명사, 부사만 남김
            words.append(word)

    words_str = ' '.join(words)
    return words_str


# df['text']를 하나씩 tokenize해서 list로 저장
tokenized_list = []

for text in df['text']:
    tokenized_list.append(tokenize_korean_text(text))

print(len(tokenized_list))
print(tokenized_list[0])

4435
남자 도대체 왜 그러는걸 까


# Vectorization & LDA

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

## vector화

In [6]:
#LDA 는 Count기반의 Vectorizer만 적용 
count_vectorizer = CountVectorizer(max_df=0.1, max_features=1000, min_df=2, ngram_range=(1,2))
    # 2개의 문서 미만으로 등장하는 단어는 제외, 전체의 10% 이상으로 자주 등장하는 단어는 제외
    # bigram도 포함

feat_vect = count_vectorizer.fit_transform(tokenized_list)
print('CountVectorizer Shape:', feat_vect.shape)

CountVectorizer Shape: (4435, 1000)


## 토픽모델링: LDA

In [7]:
lda = LatentDirichletAllocation(n_components=8)  # 토픽 수는 8개로 설정
lda.fit(feat_vect)

LatentDirichletAllocation(n_components=8)

## 토픽별 연관어 출력

In [8]:
def display_topics(model, feature_names, num_top_words):
    for topic_index, topic in enumerate(model.components_):
        print('Topic #', topic_index)

        # components_ array에서 가장 값이 큰 순으로 정렬했을 때, 그 값의 array index를 반환. 
        topic_word_indexes = topic.argsort()[::-1]  # argsort는 array의 내림차순 정렬
        top_indexes=topic_word_indexes[:num_top_words]
        
        # top_indexes대상인 index별로 feature_names에 해당하는 word feature 추출 후 join으로 concat
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])                
        print(feature_concat)

# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names_out( )를 통해 추출
feature_names = count_vectorizer.get_feature_names_out()

# Topic별 가장 연관도가 높은 word를 10개만 추출
display_topics(lda, feature_names, 10)

Topic # 0
인간 집단 정도 인간 집단 인간 인간 인물 그래도 한남 집단 인물 여성
Topic # 1
진짜 하나 여자애 중국 냄새 주제 우리 학교 때문 같이
Topic # 2
사람 짱깨 게이 없는 전라도 나라 같아 아님 자기 장애
Topic # 3
남자 결혼 아니야 무슨 그럼 해서 요즘 여자 남자 군대 그렇지
Topic # 4
사람 그냥 너무 장애인 있는 같은 나이 문제 얼굴 사는
Topic # 5
조선족 하지 그래 하고 우리 생각 하는 저런 맞아 엄마
Topic # 6
하는 그런 그렇게 잖아 한국 많이 무조건 거지 흑인 사람
Topic # 7
보면 남자 하면 하는 페미 동네 보고 하냐 해야지 되는


- 너무 그냥 싫어 있는 정도 등 예상대로 불필요한 단어들이 추출되는 것을 확인할 수 있다.

# LDA 시각화: pyLDAvis

In [9]:
!pip install pyLDAvis

In [10]:
import pyLDAvis.sklearn  # sklearn의 ldamodel에 최적화된 라이브러리

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, feat_vect, count_vectorizer)
pyLDAvis.display(vis)

C:\Users\user\anaconda3\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\user\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [11]:
# html로 저장
pyLDAvis.save_html(vis, 'topic_visualization.html')

- (참고): λ 를 1 로 설정하면 토픽 별로 가장 자주 등장하는 단어들을 우선적으로 키워드로 선택한다는 의미이며, λ를 0 에 가깝게 설정할수록 토픽 간에 차이가 많이 나는 단어를 선택한다는 의미이다.  
  
  
- '진짜', '하는', '너무' 등의 불용어아 많이 보인다.
- 이를 해결하기 위해 TF_IDF를 적용하여 LDA를 수행해보자

# 벡터화 할 때 bigram 불포함

In [21]:
#LDA 는 Count기반의 Vectorizer만 적용 
count_vectorizer = CountVectorizer(max_df=0.1, max_features=1000, min_df=2)
    # 2개의 문서 미만으로 등장하는 단어는 제외, 전체의 10% 이상으로 자주 등장하는 단어는 제외
    # bigram 불포함

feat_vect = count_vectorizer.fit_transform(tokenized_list)
print('CountVectorizer Shape:', feat_vect.shape)

lda = LatentDirichletAllocation(n_components=8)  # 토픽 수는 8개로 설정
lda.fit(feat_vect)

# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names( )를 통해 추출
feature_names = count_vectorizer.get_feature_names()

# Topic별 가장 연관도가 높은 word를 10개만 추출
display_topics(lda, feature_names, 10)

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, feat_vect, count_vectorizer)
pyLDAvis.display(vis)

CountVectorizer Shape: (4435, 1000)


C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Topic # 0
하는 사람 여자애 그런 같은 당연히 그냥 게임 아니야 없는
Topic # 1
진짜 짱깨 어디 나이 남자 생각 요즘 해서 해야 있어
Topic # 2
우리 거지 같아 맞아 중국인 아냐 하나 되는 우리나라 먹고
Topic # 3
장애인 얼굴 그럼 보면 하지 어떻게 솔직히 문제 지방 역시
Topic # 4
남자 무슨 한국 그렇게 한남 해야지 중국 조선족 화장 주제
Topic # 5
인간 하면 사는 집단 그냥 인물 잖아 아니라 사람 솔직히
Topic # 6
전라도 그런 너무 게이 냄새 아님 싫어 그래 아니냐 쓰는
Topic # 7
남자 흑인 역시 없어 결혼 페미 그래도 아니야 해도 그러게


C:\Users\user\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [22]:
# html로 저장
pyLDAvis.save_html(vis, 'topic_visualization_no_bigram.html')

# TF-IDF 적용 (bigram)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
df.head()

,Unnamed: 0,text,types,intensity
0,0,남자들은 도대체 왜 그러는걸까?,DISCRIMINATION,1.0
1,1,말하면서 침 개많이 튀길것 같아.,DISCRIMINATION,3.0
2,2,머리가 멍청해서 못읽는거아니냐?,DISCRIMINATION,3.0
3,3,"동유럽, 러시아 애들은 가난해도 사랑으로 결혼하는데 헬조선은 자본주의의 지옥 그 자체다",DISCRIMINATION,1.0
4,4,원래 남자는 365일 발정기야,DISCRIMINATION,2.0


In [34]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.1, max_features=1000, min_df=2, ngram_range=(1,2))
    # 2개의 문서 미만으로 등장하는 단어는 제외, 전체의 10% 이상으로 자주 등장하는 단어는 제외, 상위 1000개 단어만 보존
    # bigram 포함
X = tfidf_vectorizer.fit_transform(df['text'])

# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :',X.shape)

# LDA적용
lda = LatentDirichletAllocation(n_components=8)  # 토픽 수는 8개로 설정
lda.fit(X)

# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names( )를 통해 추출
feature_names = tfidf_vectorizer.get_feature_names()

# Topic별 가장 연관도가 높은 word를 10개만 추출
display_topics(lda, feature_names, 10)

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, feat_vect, tfidf_vectorizer)
pyLDAvis.display(vis)


TF-IDF 행렬의 크기 : (4435, 1000)


C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Topic # 0
그냥 하지 같은 여자들 어디 그러게 그러니까 그렇지 니가 좋겠다
Topic # 1
역시 같아 무슨 너무 너는 하는 거야 근데 남자 하면
Topic # 2
그래 남자는 네가 누가 아니면 사람 지잡대 나라 하고 없는
Topic # 3
ㅋㅋㅋ 남자가 전라도 이런 아니고 맞아 주제에 남자들은 말고 절대
Topic # 4
그런 아니야 요즘 쟤는 이렇게 아니냐 좋겠어 짱깨 하냐 여성
Topic # 5
ㅋㅋ 아냐 여자 것도 나는 몰라 하나 많이 장애인 그래서
Topic # 6
진짜 여자가 애들은 보면 그게 애들 이유가 흑인 엄청 동남아
Topic # 7
여자는 원래 싫어 이래서 그럼 말이 무조건 어떻게 솔직히 여자들이


C:\Users\user\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [35]:
# html로 저장
pyLDAvis.save_html(vis, 'topic_visualization_tfidf.html')

- 확실히 tf-idf 적용 불용어는 조금 준게 보이지만 '진짜', '역시', 같아' 등 아직 존재함을 알 수 있다. 두가지 방법으로 접근해보자
    - tf-idf 파라미터 조정
    - 순위권에 있는 불용어를 직접 제거

##  TF-IDF 적용 - 파라미터 조정(max_df)

#### max_df = 0.15

In [36]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.15, max_features=1000, min_df=2, ngram_range=(1,2))
    # 2개의 문서 미만으로 등장하는 단어는 제외, 전체의 15% 이상으로 자주 등장하는 단어는 제외, 상위 1000개 단어만 보존
    # bigram 포함
X = tfidf_vectorizer.fit_transform(df['text'])

# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :',X.shape)

# LDA적용
lda = LatentDirichletAllocation(n_components=8)  # 토픽 수는 8개로 설정
lda.fit(X)

# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names( )를 통해 추출
feature_names = tfidf_vectorizer.get_feature_names()

# Topic별 가장 연관도가 높은 word를 10개만 추출
display_topics(lda, feature_names, 10)

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, feat_vect, tfidf_vectorizer)
pyLDAvis.display(vis)



TF-IDF 행렬의 크기 : (4435, 1000)


C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Topic # 0
여자가 그런 전라도 그럼 무슨 아니 거야 아니고 이래서 이런
Topic # 1
그냥 주제에 하고 솔직히 그렇게 아니라 있는 텐데 혹시 보고
Topic # 2
ㅋㅋㅋ 남자는 애들은 요즘 있어 그래 하면 아냐 예쁜 근데
Topic # 3
진짜 아니야 너무 말고 여자들이 절대 ㅋㅋ 어차피 지잡대 내가
Topic # 4
여자는 원래 없어 맞아 너는 사람 쟤는 그래서 아님 여성
Topic # 5
남자가 말이 여자 당연히 무조건 누가 않아 여자는 쓰는 저렇게
Topic # 6
역시 어떻게 같은 남자들은 같이 많이 거지 애들이 나는 나이
Topic # 7
ㅋㅋ 같아 몰라 보면 싫어 하는 좋겠어 그러니까 봐도 이유가


C:\Users\user\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [37]:
# html로 저장
pyLDAvis.save_html(vis, 'topic_visualization_tfidf_maxdf_15.html')

#### max_df = 0.2

In [38]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.2, max_features=1000, min_df=2, ngram_range=(1,2))
    # 2개의 문서 미만으로 등장하는 단어는 제외, 전체의 20% 이상으로 자주 등장하는 단어는 제외, 상위 1000개 단어만 보존
    # bigram 포함
X = tfidf_vectorizer.fit_transform(df['text'])

# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :',X.shape)

# LDA적용
lda = LatentDirichletAllocation(n_components=8)  # 토픽 수는 8개로 설정
lda.fit(X)

# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names( )를 통해 추출
feature_names = tfidf_vectorizer.get_feature_names()

# Topic별 가장 연관도가 높은 word를 10개만 추출
display_topics(lda, feature_names, 10)

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, feat_vect, tfidf_vectorizer)
pyLDAvis.display(vis)



TF-IDF 행렬의 크기 : (4435, 1000)


C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Topic # 0
ㅋㅋㅋ 그냥 여자 같이 아니야 사람 싫어 사람들은 그래도 니가
Topic # 1
여자가 그런 무슨 내가 무조건 네가 누가 몰라 어디 것도
Topic # 2
ㅋㅋ 애들은 솔직히 나는 많이 예쁜 아니냐 애들이 없는 엄청
Topic # 3
여자는 그래 하면 여자들이 여자가 근데 흑인 한국 다들 완전
Topic # 4
역시 원래 같은 이렇게 그럼 남자는 이래서 보면 장애인 하지
Topic # 5
전라도 맞아 말이 거야 그렇게 아니면 말고 여자들 뭐가 이유가
Topic # 6
같아 남자가 아냐 어떻게 없어 저런 애들 하는 여자가 아니
Topic # 7
진짜 너무 하고 요즘 아니고 너는 거지 쟤는 그러게 하나


C:\Users\user\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [39]:
# html로 저장
pyLDAvis.save_html(vis, 'topic_visualization_tfidf_maxdf_20.html')

#### max_df = 0.3

In [40]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.3, max_features=1000, min_df=2, ngram_range=(1,2))
    # 2개의 문서 미만으로 등장하는 단어는 제외, 전체의 30% 이상으로 자주 등장하는 단어는 제외, 상위 1000개 단어만 보존
    # bigram 포함
X = tfidf_vectorizer.fit_transform(df['text'])

# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :',X.shape)

# LDA적용
lda = LatentDirichletAllocation(n_components=8)  # 토픽 수는 8개로 설정
lda.fit(X)

# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names( )를 통해 추출
feature_names = tfidf_vectorizer.get_feature_names()

# Topic별 가장 연관도가 높은 word를 10개만 추출
display_topics(lda, feature_names, 10)

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, feat_vect, tfidf_vectorizer)
pyLDAvis.display(vis)



TF-IDF 행렬의 크기 : (4435, 1000)


C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Topic # 0
진짜 그냥 여자가 아냐 그게 누가 근데 지잡대 짱깨 공부
Topic # 1
남자가 거지 솔직히 거야 남자는 무조건 그래 하지 말고 것도
Topic # 2
그런 너무 우리 어떻게 여자 보면 너는 애들 싫어 아니
Topic # 3
역시 ㅋㅋㅋ 같아 전라도 남자 이런 아니고 같이 사람 같은
Topic # 4
여자는 애들은 주제에 말이 아니라 없어 그러니까 그래서 자고로 안돼
Topic # 5
그럼 않아 그렇지 나라 많이 내가 사람들 엄청 하면 다들
Topic # 6
아니야 맞아 그래도 여자가 아니냐 남자들 머리가 니가 저렇게 네가
Topic # 7
ㅋㅋ 원래 그렇게 요즘 어디 여자들 좋겠다 남자들은 애들이 아니면


C:\Users\user\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [41]:
# html로 저장
pyLDAvis.save_html(vis, 'topic_visualization_tfidf_maxdf_30.html')

- 파라미터 조정 만으로는 한계가 있는 것 같다.
- 불용어 사전을 생성하여 직접 지워보자

In [19]:
korean_stopwords_path = "./korean_stopwords.txt"
# 텍스트 파일을 오픈합니다.
with open(korean_stopwords_path, encoding='utf-8') as f:
	stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

In [42]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.1, max_features=1000, min_df=2, ngram_range=(1,2))
    # 2개의 문서 미만으로 등장하는 단어는 제외, 전체의 10% 이상으로 자주 등장하는 단어는 제외, 상위 1000개 단어만 보존
    # bigram 포함
X = tfidf_vectorizer.fit_transform(df['text'])

# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :',X.shape)

# LDA적용
lda = LatentDirichletAllocation(n_components=8)  # 토픽 수는 8개로 설정
lda.fit(X)

# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names( )를 통해 추출
feature_names = tfidf_vectorizer.get_feature_names_out()

# Topic별 가장 연관도가 높은 word를 10개만 추출
display_topics(lda, feature_names, 10)

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, feat_vect, tfidf_vectorizer)
pyLDAvis.display(vis)



TF-IDF 행렬의 크기 : (4435, 1000)
Topic # 0
원래 같이 말이 그게 남자는 아니면 사람 아니고 무슨 여자가
Topic # 1
역시 같아 애들은 아냐 사는 하나 아님 머리가 텐데 짱깨
Topic # 2
여자는 거지 보면 싫어 하지 주제에 몰라 그렇게 요즘 것도
Topic # 3
그런 여자들은 그냥 그럼 이래서 있어 좋겠다 그러니까 하는 여자는
Topic # 4
진짜 ㅋㅋㅋ 전라도 너무 이런 같은 아니냐 네가 해서 그러게
Topic # 5
애들이 아니라 많이 너는 이제 애들 그래서 있지 예쁜 봐도
Topic # 6
ㅋㅋ 여자 남자가 아니야 아니 남자들은 이렇게 그렇지 당연히 하고
Topic # 7
어떻게 여자가 없어 거야 남자 솔직히 그래 그래도 보고 여성


C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\user\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [43]:
# html로 저장
pyLDAvis.save_html(vis, 'topic_visualization_tfidf_erase_stopwords.html')